https://www.philschmid.de/fsdp-qlora-llama3

In [24]:
%env CUDA_VISIBLE_DEVICES=1,2

env: CUDA_VISIBLE_DEVICES=1,2


In [ ]:
# this is original versions
#!pip install numpy==1.26.4
#!pip install "torch==2.2.2" tensorboard
#!pip install --upgrade "transformers==4.40.0" "datasets==2.18.0" "accelerate==0.29.3" 
#!pip install --upgrade "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.22.2" "trl==0.8.6" "peft==0.10.0"

In [37]:
#!pip install --upgrade huggingface_hub
#!pip install torch transformers datasets
#!pip install accelerate evaluate bitsandbytes trl peft tensorboardx
!pip list | grep "torch\|transformers\|huggingface-hub\|datasets\|accelerate\|evaluate\|trl\|peft\|bitsandbytes\|tensorboardx"

accelerate                0.33.0
bitsandbytes              0.43.3
datasets                  2.20.0
evaluate                  0.4.2
huggingface-hub           0.24.5
peft                      0.12.0
torch                     2.4.0
torchvision               0.19.0
transformers              4.44.0
trl                       0.9.6


In [3]:
#!huggingface-cli login --token "your_token"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/kotech/.cache/huggingface/token
Login successful


In [32]:
from datasets import load_dataset
 
# Convert dataset to OAI messages
system_message = """You are Llama, an AI assistant created by Philipp to be helpful and honest. Your knowledge spans a wide range of topics, allowing you to engage in substantive conversations and provide analysis on complex subjects."""
 
def create_conversation(sample):
    if sample["messages"][0]["role"] == "system":
        return sample
    else:
      sample["messages"] = [{"role": "system", "content": system_message}] + sample["messages"]
      return sample
 
# Load dataset from the hub
dataset = load_dataset("HuggingFaceH4/no_robots")
 
# Add system message to each conversation
columns_to_remove = list(dataset["train"].features)
columns_to_remove.remove("messages")
dataset = dataset.map(create_conversation, remove_columns=columns_to_remove,batched=False)
 
# Filter out conversations which are corrupted with wrong turns, keep which have even number of turns after adding system message
dataset["train"] = dataset["train"].filter(lambda x: len(x["messages"][1:]) % 2 == 0)
dataset["test"] = dataset["test"].filter(lambda x: len(x["messages"][1:]) % 2 == 0)
 
# save datasets to disk
#dataset["train"].to_json("train_dataset.json", orient="records", force_ascii=False)
#dataset["test"].to_json("test_dataset.json", orient="records", force_ascii=False)

In [33]:
dataset['train']

Dataset({
    features: ['messages'],
    num_rows: 9485
})

In [23]:
%%writefile llama_3_70b_fsdp_qlora.yaml
# script parameters
model_id: "meta-llama/Meta-Llama-3-70b" # Hugging Face model id
dataset_path: "."                      # path to dataset
max_seq_length:  3072 # 2048              # max sequence length for model and packing of the dataset
# training parameters
output_dir: "./llama-3-70b-hf-no-robot" # Temporary output directory for model checkpoints
report_to: "tensorboard"               # report metrics to tensorboard
learning_rate: 0.0002                  # learning rate 2e-4
lr_scheduler_type: "constant"          # learning rate scheduler
num_train_epochs: 3                    # number of training epochs
per_device_train_batch_size: 1         # batch size per device during training
per_device_eval_batch_size: 1          # batch size for evaluation
gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
optim: adamw_torch                     # use torch adamw optimizer
logging_steps: 10                      # log every 10 steps
save_strategy: epoch                   # save checkpoint every epoch
eval_strategy: epoch             # evaluate every epoch
max_grad_norm: 0.3                     # max gradient norm
warmup_ratio: 0.03                     # warmup ratio
bf16: true                             # use bfloat16 precision
tf32: true                             # use tf32 precision
#gradient_checkpointing: true           # use gradient checkpointing to save memory
# FSDP parameters: https://huggingface.co/docs/transformers/main/en/fsdp
fsdp: "full_shard auto_wrap offload" # remove offload if enough GPU memory
fsdp_config:
  activation_checkpointing: True
  backward_prefetch: "backward_pre"
  forward_prefetch: "false"
  use_orig_params: "false"
  cpu_ram_efficient_loading: "true"

Overwriting llama_3_70b_fsdp_qlora.yaml


In [ ]:
%%writefile llama_3_8b_fsdp_qlora.yaml
# script parameters
model_id: "meta-llama/Meta-Llama-3-8b" # Hugging Face model id
dataset_path: "."                      # path to dataset
max_seq_length:  3072 # 2048              # max sequence length for model and packing of the dataset
# training parameters
output_dir: "./llama-3-8b-hf-no-robot" # Temporary output directory for model checkpoints
report_to: "tensorboard"               # report metrics to tensorboard
learning_rate: 0.0002                  # learning rate 2e-4
lr_scheduler_type: "constant"          # learning rate scheduler
num_train_epochs: 3                    # number of training epochs
per_device_train_batch_size: 1         # batch size per device during training
per_device_eval_batch_size: 1          # batch size for evaluation
gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
optim: adamw_torch                     # use torch adamw optimizer
logging_steps: 10                      # log every 10 steps
save_strategy: epoch                   # save checkpoint every epoch
eval_strategy: epoch             # evaluate every epoch
max_grad_norm: 0.3                     # max gradient norm
warmup_ratio: 0.03                     # warmup ratio
bf16: true                             # use bfloat16 precision
tf32: true                             # use tf32 precision
#gradient_checkpointing: true           # use gradient checkpointing to save memory
# FSDP parameters: https://huggingface.co/docs/transformers/main/en/fsdp
fsdp: "full_shard auto_wrap offload" # remove offload if enough GPU memory
fsdp_config:
  activation_checkpointing: True
  backward_prefetch: "backward_pre"
  forward_prefetch: "false"
  use_orig_params: "false"
  cpu_ram_efficient_loading: "true"

In [15]:
!wget https://github.com/philschmid/deep-learning-pytorch-huggingface/raw/main/training/scripts/run_fsdp_qlora.py

--2024-08-09 14:27:50--  https://github.com/philschmid/deep-learning-pytorch-huggingface/raw/main/training/scripts/run_fsdp_qlora.py
github.com (github.com) 해석 중... 20.200.245.247
다음으로 연결 중: github.com (github.com)|20.200.245.247|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://raw.githubusercontent.com/philschmid/deep-learning-pytorch-huggingface/main/training/scripts/run_fsdp_qlora.py [따라감]
--2024-08-09 14:27:50--  https://raw.githubusercontent.com/philschmid/deep-learning-pytorch-huggingface/main/training/scripts/run_fsdp_qlora.py
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 6302 (6.2K) [text/plain]
저장 위치: ‘run_fsdp_qlora.py’

run_fsdp_qlora.py   100%[===================>]   6.15K  --.-KB/s    / 0s       

2024-08-09 14:27:51 (39.1 MB/s) - ‘run_fsdp_qlora.py’ 

In [16]:
%env CUDA_VISIBLE_DEVICES=1,2
#!ACCELERATE_USE_FSDP=1 FSDP_CPU_RAM_EFFICIENT_LOADING=1 torchrun --nproc_per_node=4 ./scripts/run_fsdp_qlora.py --config llama_3_70b_fsdp_qlora.yaml
!OMP_NUM_THREADS=1 ACCELERATE_USE_FSDP=1 torchrun --nproc_per_node=2 --master_port=25678 ./run_fsdp_qlora.py --config llama_3_70b_fsdp_qlora.yaml

W0809 14:28:01.735000 140132300277568 torch/distributed/run.py:779] 
W0809 14:28:01.735000 140132300277568 torch/distributed/run.py:779] *****************************************
W0809 14:28:01.735000 140132300277568 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0809 14:28:01.735000 140132300277568 torch/distributed/run.py:779] *****************************************
Traceback (most recent call last):
  File "/home/kotech/venv-torch/bin/torchrun", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/home/kotech/venv-torch/lib/python3.11/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 348, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/home/kotech/venv-torch/lib/python3.11/site-packages/torch/distributed/run.py